In [ ]:
#-----------------------------------IMPORTS-----------------------------------
import boto3
import sagemaker
import sagemaker.session

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
default_bucket = sagemaker_session.default_bucket()
model_package_group_name = f"AbaloneModelPackageGroupName"

In [ ]:
username = "[ENTER-NAME]"

In [ ]:
#-----------------------------------UPLOAD DATA TO S3-----------------------------------
local_path = "data/abalone-dataset.csv"

base_uri = f"s3://{default_bucket}/abalone"
input_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=local_path, 
    desired_s3_uri=base_uri,
)
print(input_data_uri)

## 🏌️‍♂️ Exercise 5
Add a [parameter](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-parameters.html) called `instance_type_training` which defines which AWS EC2 instance to use for the training job. A possible default value is `ml.c4.xlarge`.

In [ ]:
#-----------------------------------PARAMETERS-----------------------------------
from sagemaker.workflow.parameters import ParameterString

instance_type_training = ... # fill in here

In [ ]:
#-----------------------------------PREPROCESSING-----------------------------------
from sagemaker.sklearn.processing import SKLearnProcessor


framework_version = "0.23-1"

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    base_job_name="sklearn-abalone-process",
    role=role,
)

from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep
    

step_process = ProcessingStep(
    name="AbaloneProcess",
    processor=sklearn_processor,
    inputs=[
      ProcessingInput(source=input_data_uri, destination="/opt/ml/processing/input"),  
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test")
    ],
    code="abalone/preprocessing.py",
)

In [ ]:
#-----------------------------------TRAINING-----------------------------------
from sagemaker.estimator import Estimator

model_path = f"s3://{default_bucket}/AbaloneTrain"

image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.0-1",
    py_version="py3",
    instance_type="ml.c4.xlarge"
)
xgb_train = Estimator(
    image_uri=image_uri,
    instance_type=instance_type_training,
    instance_count=1,
    output_path=model_path,
    role=role,
)
xgb_train.set_hyperparameters(
    objective="reg:linear",
    num_round=50,
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.7,
    silent=0
)

from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep


step_train = TrainingStep(
    name="AbaloneTrain",
    estimator=xgb_train,
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv"
        )
    },
)

In [ ]:
#-----------------------------------EVALUATION-----------------------------------
from sagemaker.processing import ScriptProcessor


script_eval = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type="ml.m5.xlarge",
    instance_count=1,
    base_job_name="script-abalone-eval",
    role=role,
)

from sagemaker.workflow.properties import PropertyFile


evaluation_report = PropertyFile(
    name="EvaluationReport",
    output_name="evaluation",
    path="evaluation.json"
)


step_eval = ProcessingStep(
    name="AbaloneEval",
    processor=script_eval,
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs[
                "test"
            ].S3Output.S3Uri,
            destination="/opt/ml/processing/test"
        )
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code="abalone/evaluation.py",
    property_files=[evaluation_report],
)

In [ ]:
#-----------------------------------REGISTER MODEL-----------------------------------
from sagemaker.model_metrics import MetricsSource, ModelMetrics 
from sagemaker.workflow.step_collections import RegisterModel


model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        ),
        content_type="application/json"
    )
)
step_register = RegisterModel(
    name="AbaloneRegisterModel",
    estimator=xgb_train,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name=f"{model_package_group_name}-{username}",
    approval_status="PendingManualApproval",
    model_metrics=model_metrics
)

In [ ]:
#-----------------------------------CONDITIONAL STEP-----------------------------------
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet


cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="regression_metrics.mse.value"
    ),
    right=6.0
)

step_cond = ConditionStep(
    name="AbaloneMSECond",
    conditions=[cond_lte],
    if_steps=[step_register],
    else_steps=[], 
)

## 🏌️‍♂️ Exercise 6
Add the parameters to the `Pipeline`, so the pipeline is aware of the parameters that were passed in.

In [ ]:
#-----------------------------------DEFINE PIPELINE-----------------------------------
from sagemaker.workflow.pipeline import Pipeline


pipeline_name = f"AbalonePipelineWithParameters-{username}"
pipeline = Pipeline(
    name=pipeline_name,
    ..., # fill in parameters here
    steps=[step_process, step_train, step_eval, step_cond],
)

In [ ]:
# import json

# json.loads(pipeline.definition())

In [ ]:
#-----------------------------------SUBMIT PIPELINE-----------------------------------
pipeline.upsert(role_arn=role)

In [ ]:
#-----------------------------------RUN PIPELINE-----------------------------------
execution = pipeline.start()